In [ ]:
# Install required packages
!pip install fastapi uvicorn sqlalchemy alembic pydantic pyjwt psycopg2-binary python-multipart passlib[bcrypt] pytest

# Output confirmation message
print("Environment")

In [ ]:
import os
import shutil

# Define the project structure
project_dirs = [
    "app",
    "app/api",
    "app/api/endpoints",
    "app/core",
    "app/db",
    "app/models",
    "app/schemas",
    "app/services",
    "tests",
    "tests/api",
    "tests/services",
    "alembic",
    "alembic/versions"
]

# Create the directory structure
for dir_path in project_dirs:
    os.makedirs(dir_path, exist_ok=True)
    print(f"Created directory: {dir_path}")

print("Project structure created successfully");

In [ ]:
# app/core/config.py
config_content = '''
from pydantic import BaseSettings
from typing import Optional, Dict, Any, List

class Settings(BaseSettings):
    PROJECT_NAME: str = "CyberEd Pro"
    API_V1_STR: str = "/api"

    # SECURITY
    SECRET_KEY: str = "CHANGE_THIS_TO_A_SECURE_KEY"  # For production, use a secure key!
    ACCESS_TOKEN_EXPIRE_MINUTES: int = 60 * 24 * 8  # 8 days

    # DATABASE
    POSTGRES_SERVER: str = "localhost"
    POSTGRES_USER: str = "postgres"
    POSTGRES_PASSWORD: str = "postgres"
    POSTGRES_DB: str = "cybered_pro"
    SQLALCHEMY_DATABASE_URI: Optional[str] = None

    @property
    def get_database_url(self) -> str:
        """Generate the database URL."""
        return f"postgresql://{self.POSTGRES_USER}:{self.POSTGRES_PASSWORD}@{self.POSTGRES_SERVER}/{self.POSTGRES_DB}"

    class Config:
        case_sensitive = True
        env_file = ".env"

settings = Settings()
settings.SQLALCHEMY_DATABASE_URI = settings.get_database_url
'''

with open("app/core/config.py", "w") as f:
    f.write(config_content)

print("Database configuration created!")

In [ ]:
# app/db/base.py
db_base_content = '''
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

from app.core.config import settings

engine = create_engine(settings.SQLALCHEMY_DATABASE_URI, pool_pre_ping=True)
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)

Base = declarative_base()

# Dependency
def get_db():
    db = SessionLocal()
    try:
        yield db
    finally:
        db.close()
'''

with open("app/db/base.py", "w") as f:
    f.write(db_base_content)

print("Database base setup create

In [ ]:
# app/models/user.py
user_model_content = '''
from sqlalchemy import Boolean, Column, Integer, String, DateTime
from sqlalchemy.sql import func
from app.db.base import Base

class User(Base):
    __tablename__ = "users"

    id = Column(Integer, primary_key=True, index=True)
    email = Column(String, unique=True, index=True, nullable=False)
    first_name = Column(String, nullable=False)
    last_name = Column(String, nullable=False)
    hashed_password = Column(String, nullable=False)
    role = Column(String, nullable=False, default="student")  # student, instructor, admin
    is_active = Column(Boolean(), default=True)
    created_at = Column(DateTime(timezone=True), server_default=func.now())
    updated_at = Column(DateTime(timezone=True), onupdate=func.now())
'''

with open("app/models/user.py", "w") as f:
    f.write(user_model_content)

print("User model created!")

In [ ]:
# app/models/course.py
course_model_content = '''
from sqlalchemy import Column, Integer, String, Text, DateTime, ForeignKey
from sqlalchemy.sql import func
from app.db.base import Base

class Course(Base):
    __tablename__ = "courses"

    id = Column(Integer, primary_key=True, index=True)
    title = Column(String, nullable=False)
    description = Column(Text, nullable=True)
    instructor_id = Column(Integer, ForeignKey("users.id"), nullable=False)
    certification_type = Column(String, nullable=True)  # Security+, CEH, CISSP, etc.
    difficulty_level = Column(String, nullable=False)  # beginner, intermediate, advanced
    estimated_duration = Column(Integer, nullable=True)  # in hours
    created_at = Column(DateTime(timezone=True), server_default=func.now())
    updated_at = Column(DateTime(timezone=True), onupdate=func.now())

class Module(Base):
    __tablename__ = "modules"

    id = Column(Integer, primary_key=True, index=True)
    course_id = Column(Integer, ForeignKey("courses.id"), nullable=False)
    title = Column(String, nullable=False)
    description = Column(Text, nullable=True)
    order_index = Column(Integer, nullable=False)
    created_at = Column(DateTime(timezone=True), server_default=func.now())
    updated_at = Column(DateTime(timezone=True), onupdate=func.now())

class Lesson(Base):
    __tablename__ = "lessons"

    id = Column(Integer, primary_key=True, index=True)
    module_id = Column(Integer, ForeignKey("modules.id"), nullable=False)
    title = Column(String, nullable=False)
    content_type = Column(String, nullable=False)  # text, video, interactive
    content = Column(Text, nullable=False)
    order_index = Column(Integer, nullable=False)
    created_at = Column(DateTime(timezone=True), server_default=func.now())
    updated_at = Column(DateTime(timezone=True), onupdate=func.now())
'''

with open("app/models/course.py", "w") as f:
    f.write(course_model_content)

print("Course models created!")

In [ ]:
# app/schemas/user.py
user_schema_content = '''
from typing import Optional, List
from pydantic import BaseModel, EmailStr, validator
import re

class UserBase(BaseModel):
    email: EmailStr
    first_name: str
    last_name: str
    role: str

    @validator("role")
    def validate_role(cls, v):
        allowed_roles = ["student", "instructor", "admin"]
        if v not in allowed_roles:
            raise ValueError(f"Role must be one of {allowed_roles}")
        return v

class UserCreate(UserBase):
    password: str

    @validator("password")
    def password_strength(cls, v):
        if len(v) < 8:
            raise ValueError("Password must be at least 8 characters")
        if not re.search(r"[A-Z]", v):
            raise ValueError("Password must contain at least one uppercase letter")
        if not re.search(r"[a-z]", v):
            raise ValueError("Password must contain at least one lowercase letter")
        if not re.search(r"[0-9]", v):
            raise ValueError("Password must contain at least one number")
        return v

class UserUpdate(BaseModel):
    email: Optional[EmailStr] = None
    first_name: Optional[str] = None
    last_name: Optional[str] = None

class UserInDBBase(UserBase):
    id: int
    is_active: bool
    created_at: str

    class Config:
        orm_mode = True

class User(UserInDBBase):
    pass

class UserInDB(UserInDBBase):
    hashed_password: str
'''

with open("app/schemas/user.py", "w") as f:
    f.write(user_schema_content)

# app/schemas/course.py
course_schema_content = '''
from typing import Optional, List
from pydantic import BaseModel, validator

class CourseBase(BaseModel):
    title: str
    description: Optional[str] = None
    certification_type: Optional[str] = None
    difficulty_level: str
    estimated_duration: Optional[int] = None

    @validator("difficulty_level")
    def validate_difficulty(cls, v):
        allowed_levels = ["beginner", "intermediate", "advanced"]
        if v not in allowed_levels:
            raise ValueError(f"Difficulty level must be one of {allowed_levels}")
        return v

class CourseCreate(CourseBase):
    instructor_id: int

class CourseUpdate(BaseModel):
    title: Optional[str] = None
    description: Optional[str] = None
    certification_type: Optional[str] = None
    difficulty_level: Optional[str] = None
    estimate